In [1]:
# Imports
import pandas as pd
import numpy as np
from decouple import config
import plotly.graph_objects as go
from datetime import datetime
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Extracting data from GSheets

In [2]:
# Connect to GSheets
sheet_id = config("SHEET_ID")
sheet_name = "Applications"
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
applications_raw = pd.read_csv(sheet_url)
applications_raw

,S/N,Company,Position,Status 1,Status 2,Status 3,Status 4,Date Applied,Source,Link,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1,Thales,Software Engineer Intern,Rejected,NaN,NaN,NaN,27/03/2024,LinkedIn,https://careers.thalesgroup.com/global/en/job/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Garda Capital Partners,Software Engineer Intern (Python),NaN,NaN,NaN,NaN,27/03/2024,LinkedIn,https://boards.greenhouse.io/gardacp/jobs/4174...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Hudson River Trading,Software Engineer Intern,Rejected,NaN,NaN,NaN,27/03/2024,LinkedIn,https://boards.greenhouse.io/wehrtyou/jobs/532...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ByteDance,Data Center Development Intern (International ...,Technical Assessment,Rejected,NaN,NaN,27/03/2024,LinkedIn,https://jobs.bytedance.com/en/position/7259704...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,FlexoSense,Intern - Software development - app/dashboard,NaN,NaN,NaN,NaN,27/03/2024,NUS TalentConnect,https://nus-csm.symplicity.com/students/app/jo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Cleaning
applications = applications_raw.dropna(how='all', axis=1)
applications.drop(columns=['S/N', 'Company', 'Position', 'Date Applied', 'Link'], inplace=True)
applications.dropna(how='all', axis=0, inplace=True)

applications

/var/folders/lh/z5tbmkb917q03kyx2d6ncdzr0000gn/T/ipykernel_18911/4038210855.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  applications.drop(columns=['S/N', 'Company', 'Position', 'Date Applied', 'Link'], inplace=True)
/var/folders/lh/z5tbmkb917q03kyx2d6ncdzr0000gn/T/ipykernel_18911/4038210855.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  applications.dropna(how='all', axis=0, inplace=True)


,Status 1,Status 2,Status 3,Status 4,Source
0,Rejected,NaN,NaN,NaN,LinkedIn
1,NaN,NaN,NaN,NaN,LinkedIn
2,Rejected,NaN,NaN,NaN,LinkedIn
3,Technical Assessment,Rejected,NaN,NaN,LinkedIn
4,NaN,NaN,NaN,NaN,NUS TalentConnect
...,...,...,...,...,...
159,NaN,NaN,NaN,NaN,LinkedIn
160,NaN,NaN,NaN,NaN,LinkedIn
161,NaN,NaN,NaN,NaN,LinkedIn
162,NaN,NaN,NaN,NaN,LinkedIn


In [4]:
# Lists of unique values for various phases
job_sources = applications['Source'].unique().tolist()

status_stages = [stage for stage in applications.columns if stage.startswith('Status')]
for i in range(len(status_stages)):
    if i == 0:
        statuses = applications[status_stages[i]].unique().tolist()
    else:
        statuses.extend(applications[status_stages[i]].unique().tolist())
statuses = list(set(statuses))
statuses.remove(np.nan)

intermediate_statuses = statuses.copy()
to_remove = ['Rejected', 'DNF', 'Offered', 'Accepted', 'Declined']
for item in to_remove:
    if item in intermediate_statuses:
        intermediate_statuses.remove(item)

unique_nodes = ['Applications', 'No reply']
unique_nodes += job_sources + statuses

node_value_counts = {node: 0 for node in unique_nodes}
job_sources_count = applications['Source'].value_counts().to_dict()
for source in job_sources_count:
    node_value_counts[source] += job_sources_count[source]
for stage in status_stages: 
    status_count = applications[stage].value_counts().to_dict()
    for status in status_count:
        node_value_counts[status] += status_count[status]
node_value_counts['Applications'] = len(applications)
node_value_counts['No reply'] = len(applications[applications['Status 1'].isna() & applications['Status 2'].isna()])

unique_nodes_with_values = [node + ': ' + str(node_value_counts[node]) for node in node_value_counts]

print(job_sources)
print(statuses)
print(intermediate_statuses)
print(unique_nodes)
print(unique_nodes_with_values)

['LinkedIn', 'NUS TalentConnect', 'Indeed', 'Friends', 'NUS SWS', 'Email', 'NUS IAAS']
['Accepted', 'Online Interview', 'DNF', 'Offered', 'On-site Interview', 'Rejected', 'Declined', 'Technical Assessment']
['Online Interview', 'On-site Interview', 'Technical Assessment']
['Applications', 'No reply', 'LinkedIn', 'NUS TalentConnect', 'Indeed', 'Friends', 'NUS SWS', 'Email', 'NUS IAAS', 'Accepted', 'Online Interview', 'DNF', 'Offered', 'On-site Interview', 'Rejected', 'Declined', 'Technical Assessment']
['Applications: 164', 'No reply: 127', 'LinkedIn: 48', 'NUS TalentConnect: 21', 'Indeed: 78', 'Friends: 2', 'NUS SWS: 8', 'Email: 2', 'NUS IAAS: 5', 'Accepted: 1', 'Online Interview: 5', 'DNF: 1', 'Offered: 2', 'On-site Interview: 1', 'Rejected: 33', 'Declined: 1', 'Technical Assessment: 5']


In [5]:
# Converting data to format for plotly but still in human-readable format

# Make df for sankey graph
sankey_df = pd.DataFrame(columns=['source', 'target', 'value'])

# Applications -> Source
application_sources = applications['Source'].value_counts()
for application_source in application_sources.index:
    source = application_source
    value = application_sources[application_source]
    sankey_df = sankey_df._append({'source': 'Applications', 'target': source, 'value': value}, ignore_index=True)

# Source -> Status 1
application_statuses_1 = {}
for index, row in applications.iterrows():
    if pd.notna(row['Status 1']):
        status_update_1 = row['Source'], row['Status 1']
        if status_update_1 in application_statuses_1:
            application_statuses_1[status_update_1] += 1
        else:
            application_statuses_1[status_update_1] = 1
    else:
        no_update_status = row['Source'], 'No reply'
        if no_update_status in application_statuses_1:
            application_statuses_1[no_update_status] += 1
        else:
            application_statuses_1[no_update_status] = 1

for status_update_1 in application_statuses_1:
    sankey_df = sankey_df._append({'source': status_update_1[0], 'target': status_update_1[1], 'value': application_statuses_1[status_update_1]}, ignore_index=True)

# Status 1 -> Status 2
# application_statuses_2 = {}
# for index, row in applications.iterrows():
#     if pd.notna(row['Status 2']):
#         flow = row['Status 1'], row['Status 2']
#         if flow in application_statuses_2:
#             application_statuses_2[flow] += 1
#         else:
#             application_statuses_2[flow] = 1

# for flow in application_statuses_2:
#     sankey_df = sankey_df._append({'source': flow[0], 'target': flow[1], 'value': application_statuses_2[flow]}, ignore_index=True)

# Status 1 -> Status N
for i in range(len(status_stages)):
    if i != 0:
        application_statuses_i = {}
        for index, row in applications.iterrows():
            if pd.notna(row[status_stages[i]]):
                flow = row[status_stages[i-1]], row[status_stages[i]]
                if flow in application_statuses_i:
                    application_statuses_i[flow] += 1
                else:
                    application_statuses_i[flow] = 1
        
        for flow in application_statuses_i:
            sankey_df = sankey_df._append({'source': flow[0], 'target': flow[1], 'value': application_statuses_i[flow]}, ignore_index=True)

sankey_df

,source,target,value
0,Applications,Indeed,78
1,Applications,LinkedIn,48
2,Applications,NUS TalentConnect,21
3,Applications,NUS SWS,8
4,Applications,NUS IAAS,5
5,Applications,Friends,2
6,Applications,Email,2
7,LinkedIn,Rejected,13
8,LinkedIn,No reply,33
9,LinkedIn,Technical Assessment,2


In [6]:
## Creating color references for nodes and links

# Define variables for colors for easier modification
node_blue = 'rgba(39, 125, 161, 1)'
node_yellow = 'rgba(249, 199, 79, 1)'
node_grey = 'rgba(173, 181, 189, 1)'
node_red = 'rgba(249, 65, 68, 1)'
node_green = 'rgba(67, 170, 139, 1)'
node_black = 'rgba(0, 0, 0, 1)'
link_blue = node_blue.replace('1)', '0.6)')
link_yellow = node_yellow.replace('1)', '0.6)')
link_grey = node_grey.replace('1)', '0.6)')
link_red = node_red.replace('1)', '0.6)')
link_green = node_green.replace('1)', '0.6)')
link_black = node_black.replace('1)', '0.6)')


node_colors = ['rgba(39, 125, 161, 1)'] # Applications unaccounted for but should already be in list
for source_target in unique_nodes:
    if source_target in job_sources:
        node_colors.append(node_blue) 
    elif source_target in intermediate_statuses:
        node_colors.append(node_yellow)
    elif source_target == 'No reply':
        node_colors.append(node_grey)
    elif source_target == 'Rejected':
        node_colors.append(node_red)
    elif source_target == 'DNF':
        node_colors.append(node_black)
    elif source_target in ['Offered', 'Accepted', 'Declined']:
        node_colors.append(node_green) 

link_colors = []
for index, row in sankey_df.iterrows():
    if row['target'] in job_sources:
        link_colors.append(link_blue)
    elif row['target'] in intermediate_statuses:
        link_colors.append(link_yellow)
    elif row['target'] == 'No reply':
        link_colors.append(link_grey)
    elif row['target'] == 'Rejected':
        link_colors.append(link_red)
    elif row['target'] == 'DNF':
        link_colors.append(link_black)
    elif row['target'] in ['Offered', 'Accepted', 'Declined']:
        link_colors.append(link_green)

In [7]:
## Conversion to plotly-readable format

# # Generating labels with values
# node_value_counts = sankey_df['source'].value_counts()
# node_labels = [node for node in unique_nodes]

# Assign unique number to each source/target
mapping_dict = {k: v for v, k in enumerate(unique_nodes)}

# Map the sources/targets to their unique number
sankey_df['source'] = sankey_df['source'].map(mapping_dict)
sankey_df['target'] = sankey_df['target'].map(mapping_dict)

sankey_df

,source,target,value
0,0,4,78
1,0,2,48
2,0,3,21
3,0,6,8
4,0,8,5
5,0,5,2
6,0,7,2
7,2,14,13
8,2,1,33
9,2,16,2


In [8]:
# Explicitly assigning positions for nodes
node_pos = {}
node_pos['Applications'] = (0.1, 0.5)
for job_source in job_sources:
    node_pos[job_source] = (0.3, 0.1)

node_pos['Rejected'] = (0.5, 0.9)

node_pos['Technical Assessment'] = (0.55, 0.7)

node_pos['No reply'] = (0.6, 0.1)
node_pos['On-site Interview'] = (0.6, 0.675)
node_pos['Online Interview'] = (0.6, 0.725)

node_pos['DNF'] = (0.65, 0.85)

node_pos['Offered'] = (0.8, 0.7)

node_pos['Declined'] = (0.85, 0.75)
node_pos['Accepted'] = (0.9, 0.65)

node_x_pos = [node_pos[node][0] for node in unique_nodes]
node_y_pos = [node_pos[node][1] for node in unique_nodes]


In [25]:
# Plot Sankey Diagram
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    arrangement = "snap",
    node = dict(
      pad = 20,
      thickness = 10,
      label = unique_nodes_with_values,
      color = node_colors,
      x = node_x_pos,
      y = node_y_pos
    ),
    link = dict(
      source = sankey_df['source'].to_list(),
      target = sankey_df['target'].to_list(),
      value = sankey_df['value'].to_list(),
      color = link_colors
  ))])

fig.update_layout(title_text="Dylan's Internship Applications as a Y2 CS Undergrad", title_xanchor='center', title_x=0.5, title_font_size=30, title_font_family='Helvetica',
                  font_size=14, font_family='Helvetica',
                  annotations=[dict(x=0.5, y=1.07, showarrow=False, text=f"caa {datetime.today().date().strftime('%d %b %Y')}", xref="paper", yref="paper")],
                  width=1200, height=800)
fig.show()

In [10]:
# Export to png
fig.write_image(f"data/output/Internship Applications Sankey Diagram {datetime.today().date().strftime('%d%m%y')}.png")

In [11]:
# (Optional) Export to html
# fig.write_html(f"data/output/Internship Applications Sankey Diagram {datetime.today().date().strftime('%d%m%y')}.html")